In [1]:
import os
import cv2
import json
import mediapipe as mp

In [2]:
def save_to_json(json_file_path, data):
    with open(json_file_path, 'a') as json_file:
        json.dump(data, json_file)
        json_file.write('\n')

def process_frame(frame, pose, json_file_path):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        keypoints = []
        index = 0
        for data_point in results.pose_landmarks.landmark:
            keypoints.append({
                'X': data_point.x,
                'Y': data_point.y,
                'Z': data_point.z,
                'Visibility': data_point.visibility,
                'Landmark': index
            })
            index += 1

        save_to_json(json_file_path, keypoints)

    return frame

mp_pose = mp.solutions.pose

video_folder = "seg_videos"
json_folder = "json"

# Ensure the json folder exists
if not os.path.exists(json_folder):
    os.makedirs(json_folder)

for filename in os.listdir(video_folder):
    if filename.endswith(".mp4"):
        video_path = os.path.join(video_folder, filename)
        output_path = os.path.join(json_folder, f"{os.path.splitext(filename)[0]}.json")

        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"Error: Could not open video file: {filename}")
        else:
            pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

            while cap.isOpened():
                ret, frame = cap.read()

                if not ret:
                    break

                process_frame(frame, pose, output_path)

            cap.release()
            pose.close()
